In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup as BS
import re
import glob
from io import StringIO
from scrapy import Selector

In [2]:
path = r'E:\NSS\nss_data_science\dq9-edgar-the-gators-master\data' 
all_data = glob.glob(path + "\*.htm")

In [3]:
#read_html only parse table
list_htms = [open(htm).read() for htm in all_data]

In [4]:
# i,j = 1,2
# if i ==1:
#     print(i)
# if j ==2:
#     print(j)

In [70]:
ss = '<fdsfa>'
ss.strip('<|>')

'fdsfa'

In [143]:
sel = Selector(text = list_htms[0])
target_tags = sel.xpath("//p[contains(@style, font-size) or contains(@style, font)]").extract()

target_tags

# font_size = [int(re.search(r'font.+:\s*(\d+)pt', x).group(1)) for x in target_tags if re.search(r'font.+:\s*(\d+)pt', x)]

# re.sub(r'<.+?>', '', sel.xpath("//p[contains(@style, '{}')]".format(max(font_size))).extract()[0])



['<p style="margin:0in 0in .0001pt;"><font size="2" face="Times New Roman" style="font-size:10.0pt;">\xa0</font></p>',
 '<p style="margin:0in 0in .0001pt;"><b><font size="2" face="Arial" style="font-size:10.0pt;font-weight:bold;">PROSPECTUS and</font></b></p>',
 '<p style="margin:0in 0in .0001pt;"><font size="2" face="Arial" style="font-size:1.0pt;">\xa0</font></p>',
 '<p style="margin:0in 0in .0001pt;"><b><font size="2" face="Arial" style="font-size:10.0pt;font-weight:bold;">PRICING SUPPLEMENT NO. 34</font></b></p>',
 '<p style="margin:0in 0in .0001pt;"><b><font size="2" face="Arial" style="font-size:10.0pt;font-weight:bold;">PROSPECTUS SUPPLEMENT, each</font></b></p>',
 '<p style="margin:0in 0in .0001pt;"><font size="2" face="Arial" style="font-size:1.0pt;">\xa0</font></p>',
 '<p style="margin:0in 0in .0001pt;"><b><font size="2" face="Arial" style="font-size:10.0pt;font-weight:bold;">Dated September\xa09, 2019</font></b></p>',
 '<p style="margin:0in 0in .0001pt;"><b><font size="2" fa

'Japan Bank for International Cooperation '

In [110]:
max(font_size)

22

In [75]:
re.search(r'<\/.+font.+:.+pt>', target_tags[0])

In [76]:
target_tags[0]

'<p style="margin-top:0pt; margin-bottom:0pt; font-size:10pt; font-family:Times New Roman" align="right"><b>Filed pursuant to Rule 424(b)(5)<br>Registration Number 333-217421 </b></p>'

In [23]:
sel.xpath("//p[contains(@style,'text-align:\scenter')]").extract()

[]

In [125]:
re.search(r'[A-Z].+\n', s).group(0)

'Japan\xa0Bank\xa0for\xa0International Cooperation\n'

In [106]:
#"//*[contains(@align,'center') or contains(@style,'text-align:center') \ or contains(@style,'text-align: center')]"

list_output = []

for htm in list_htms:
    
    output = dict()

    sel = Selector(text = htm)
    target_tags = sel.xpath("//p[contains(@style, font-size) or contains(@style, font)]").extract()
    texts = [re.sub(r'<.+?>', '', x) for x in target_tags]

    for text in texts:
        
        text = text.replace('\xa0', '')
        text = re.sub('(.+)\n(.+)', r'\1\2', text)
        
        #match_issuer = re.match(r'.+', text)
        match_security = re.search(r'.+due.+\d{4}', text, re.IGNORECASE)
        #match_digit_comma = re.search(r'[\d,-]', text)

#         if match_issuer and not match_digit_comma:
#             output.update({'issuer_name':match_issuer.group(0)})

        if match_security:

            output.update({'security_description' : match_security.group(0)})
            break
            
    list_output.append(output) # append the output for each file 
    
df_2 = pd.DataFrame(list_output,index=[re.search(r'data\\(.+htm)$',x).group(1) for x in all_data])

In [107]:
df_2 = df_2.reset_index()
df_2['coupon_type_code'] = ['Float' if re.search(r'float', str(x), re.IGNORECASE) else 'Fix' for x in df_2.security_description]

In [108]:
df_2

,index,security_description,coupon_type_code
0,424B2_a19-18449_4424b2.htm,"$400,000,000 Floating Rate Senior Notes Due Ju...",Float
1,424B2_a19-18449_5424b2.htm,"$400,000,000 1.950% Fixed Rate Senior Notes Du...",Fix
2,424B2_a19-18449_6424b2.htm,"$450,000,000 2.250% Fixed Rate Senior Notes Du...",Fix
3,424B2_a19-18959_1424b2.htm,"$600,000,000 3.550% Fixed Rate Notes Due 2022",Fix
4,424B2_a20-1355_3424b2.htm,"$550,000,000 2.450% Fixed Rate Senior Notes Du...",Fix
5,424B2_a20-1355_4424b2.htm,"$550,000,000 2.050% Fixed Rate Senior Notes Du...",Fix
6,424B2_d26200424b2.htm,"Due August 5, 2022",Fix
7,424B2_d795231d424b2.htm,"2.250% Medium-Term Notes, Series A, due Januar...",Fix
8,424B2_d816260d424b2.htm,"offering and sale of $500,000,000 aggregate pr...",Float
9,424B2_d833231d424b2.htm,"UST 1.375% due January31, 2025",Fix


In [192]:
df.reset_index()

,index,issuer_name,cusip_id,isin_id,original_face_amount,maturity_date,cpn_pay_frq_code
0,424B2_a19-18449_4424b2.htm,[],[24422EUZ0],[US24422EUZ05],"[ $400,000,000]","[ June13, 2022]","[ Quarterly on the 13th of March, June, Sep..."
1,424B2_a19-18449_5424b2.htm,[],[24422EVA4],[US24422EVA45],"[ $400,000,000]","[ June13, 2022]",[]
2,424B2_a19-18449_6424b2.htm,[],[24422EVB2],[US24422EVB28],"[ $450,000,000]","[ September14, 2026]","[ Semi-annually on March14 and September14, ..."
3,424B2_a19-18959_1424b2.htm,[ Ford Motor Credit Company LLC ],[34540XAB9],[US34540XAB91],"[ $600,000,000]","[ October7, 2022]",[ Semi-annually on each April7 and October7...
4,424B2_a20-1355_3424b2.htm,[],[24422EVD8],[US24422EVD83],"[ $550,000,000]","[ January9, 2030]","[ Semi-annually on January9 and July9, commenc..."
5,424B2_a20-1355_4424b2.htm,[],[24422EVC0],[US24422EVC01],"[ $550,000,000]","[ January9, 2025]","[ Semi-annually on January9 and July9, commenc..."
6,424B2_d26200424b2.htm,[Royal Bank of Canada (the “Bank”)],[78015K7F5],[US78015K7F50],"[US $300,000,000]","[August 5, 2022]","[Quarterly on the 5th of each February, May, A..."
7,424B2_d795231d424b2.htm,[],"[September, 00914AAE2]","[US00914AAD46, US00914AAE29]","[ $600,000,000, $500,000,000 Trade Date: Sept...","[ 2.250% per annum, accruing from September16,...","[ Each January15 and July15, beginning on Janu..."
8,424B2_d816260d424b2.htm,[],[02665WDE8],[],"[ $500,000,000]","[ February15, 2022, 0.0933]","[ Each February15, May15, August 15 andNovembe..."
9,424B2_d833231d424b2.htm,[],[95000U2K8],[],[],"[ February11, 2026]","[ Each February11 and August11, commencing Aug..."


In [58]:
g = pd.read_html(all_data[26])[0]

In [61]:
g.iloc[1,2]

'Japan\xa0Bank\xa0for\xa0International Cooperation (Incorporated under the Japan Bank for International Cooperation Act)  U.S.$2,250,000,000 1.750% Guaranteed Bonds Due 2023 Unconditionally and Irrevocably Guaranteed as to Payment of Principal and Interest by Japan'

In [213]:
for doc in pd.read_html(all_data[26]): # loop over the tables in the docs
    file = pd.concat([file, doc], 0, ignore_index=True) # concatnate the tables as one dataframe

file = file.dropna(axis=0, how='all').fillna('') # drop the rows containning all NAs
cols = file.columns

file['column'] = ""

for col in cols:
    file['column'] = file['column'] + file[col].astype('str') # merge all columns to the new one

file = pd.DataFrame(file.column).drop_duplicates(keep='first')

In [214]:
file[file.column.str.contains('CUSIP')].values

array([['CUSIP No.:471048 CG1471048 CG1471048 CG1CUSIP No.:471048 CG1471048 CG1471048 CG1']],
      dtype=object)

In [6]:
# # . could match everything but '\n'
# # previous pattern r'(.{0,17}[acdmi]\w+:\W*.{1,60}\d)
# for i in range(0,len(list_htms)):
#     soup = BS(list_htms[i])
#     string = soup.text.replace('\xa0','')
#     print('[{}]'.format(i) + ' :' + all_data[i][-26:])
#     print(re.findall(r'\b[ACMIPS].+[tesyNPd)\*]\s?:\W*.{1,60}\d', string))
#     print('_______________________________________________________________________')

In [7]:
check_list = []

for i in range(0,len(list_htms)):
    soup = BS(list_htms[i])
    string = soup.text.replace('\xa0','')
    string = re.sub('(.+)\n(.+)', r'\1\2', string)
    file_name = '#{}'.format(i) + ' :' + all_data[i][-26:]
    check_list.append(file_name)
    temp_list = re.findall(r'\b[ACMIPS].+:\W*.+', string)
    for x in temp_list:
        if x.find(':') <40 and len(x) < 600:
            check_list.append(x.replace('\n', ""))
    check_df = pd.DataFrame(check_list, columns=['value'])

In [92]:
#check_df.loc[check_df.value.str.contains(r'[Pp]ayment.+:')].value.str.len().max()

In [49]:
# This pattern has match groups. To actually get the groups, use str.extract.
check_df[check_df.value.str.contains(r'CUSIP|ISIN|#')].values

array([['#0 :424B2_a19-18449_4424b2.htm'],
       ['CUSIP / ISIN:  24422EUZ0 / US24422EUZ05  '],
       ['#1 :424B2_a19-18449_5424b2.htm'],
       ['CUSIP /   ISIN:  24422EVA4 /   US24422EVA45  '],
       ['#2 :424B2_a19-18449_6424b2.htm'],
       ['CUSIP / ISIN: 24422EVB2 / US24422EVB28  '],
       ['#3 :424B2_a19-18959_1424b2.htm'],
       ['CUSIP / ISIN:  34540XAB9 / US34540XAB91   '],
       ['#4 :\\424B2_a20-1355_3424b2.htm'],
       ['CUSIP / ISIN: 24422EVD8 / US24422EVD83  '],
       ['#5 :\\424B2_a20-1355_4424b2.htm'],
       ['CUSIP / ISIN: 24422EVC0 /   US24422EVC01  '],
       ['#6 :data\\424B2_d26200424b2.htm'],
       ['CUSIP / ISIN / Common     Code:78015K7F5/ US78015K7F50/ 211669616'],
       ['#7 :ta\\424B2_d795231d424b2.htm'],
       ['CUSIP / ISIN: 00914AAD4 / US00914AAD46  The2029 Notes  The following terms apply to the 2029 Notes: Principal Amount: $500,000,000 Trade Date: September9, 2019 '],
       ['CUSIP / ISIN: 00914AAE2 / US00914AAE29 Optional Redemption  The 

In [190]:
def extract_raw_info(data):
    
    list_output = []
    
    for htm in data: # loop over the htms in the list
        
        string = BS(htm).text.replace('\xa0','')
        
        string = re.sub('(.+)\n(.+)', r'\1\2', string)
        
        fields = re.findall(r'\b[ACMIPS].+:\W*.+', string)
        
        output = dict({'issuer_name': [], 'cusip_id': [], 'isin_id': [], 
                       'original_face_amount': [], 'maturity_date': [], 'cpn_pay_frq_code': []})

        # convert august to digit and remove comma
        for value in fields: # iterate the index and value for each row in the file
           
            if value.find(':') <40 and len(value) < 600:
            # match each fileds needed to be extracted out
                value = value.replace('\n', "")# remove '\n' cause . does not match it in python by default
                match_issuer = re.match(r'Issuer:(.+)', value)
                #match_security = re.search(r'([A-Z].+)[Dd]ue.+\d', value.col)
                match_cusip =  re.match(r'CUSIP.*:\W*(\w{9})', value, re.IGNORECASE)
                match_isin =  re.search(r'ISIN.*:.*(\w{12})', value, re.IGNORECASE)
                match_amount =  re.search(r'Amount.?:(.+\w)', value, re.IGNORECASE)
                match_maturity = re.search(r'Maturity.*:(.+\d{4})', value, re.IGNORECASE)
                match_frequency = re.search(r'Payment Dates.?:(.+)', value, re.IGNORECASE)

                # if matched, extract the relevant groups and update the key-value pairs in the dict of output
                if match_issuer:
                    output['issuer_name'].append(match_issuer.group(1))
        #             elif match_security:
        #                 output.update({'security_description' : match_security.group(1)})
                if match_cusip:
                    output['cusip_id'].append(match_cusip.group(1))
                if match_isin:#?? there is isin but no cusip
                    output['isin_id'].append(match_isin.group(1))
                if match_amount:
                    output['original_face_amount'].append(match_amount.group(1))
                if match_maturity:
                    output['maturity_date'].append(match_maturity.group(1))
                if match_frequency:
                    output['cpn_pay_frq_code'].append(match_frequency.group(1))
       
        list_output.append(output) # append the output for each file 
        
    return (pd.DataFrame(list_output,index=[re.search(r'data\\(.+htm)$',x).group(1) for x in all_data])) #convert to dataframe

In [191]:
df = extract_raw_info(list_htms)

In [192]:
df.reset_index()

,index,issuer_name,cusip_id,isin_id,original_face_amount,maturity_date,cpn_pay_frq_code
0,424B2_a19-18449_4424b2.htm,[],[24422EUZ0],[US24422EUZ05],"[ $400,000,000]","[ June13, 2022]","[ Quarterly on the 13th of March, June, Sep..."
1,424B2_a19-18449_5424b2.htm,[],[24422EVA4],[US24422EVA45],"[ $400,000,000]","[ June13, 2022]",[]
2,424B2_a19-18449_6424b2.htm,[],[24422EVB2],[US24422EVB28],"[ $450,000,000]","[ September14, 2026]","[ Semi-annually on March14 and September14, ..."
3,424B2_a19-18959_1424b2.htm,[ Ford Motor Credit Company LLC ],[34540XAB9],[US34540XAB91],"[ $600,000,000]","[ October7, 2022]",[ Semi-annually on each April7 and October7...
4,424B2_a20-1355_3424b2.htm,[],[24422EVD8],[US24422EVD83],"[ $550,000,000]","[ January9, 2030]","[ Semi-annually on January9 and July9, commenc..."
5,424B2_a20-1355_4424b2.htm,[],[24422EVC0],[US24422EVC01],"[ $550,000,000]","[ January9, 2025]","[ Semi-annually on January9 and July9, commenc..."
6,424B2_d26200424b2.htm,[Royal Bank of Canada (the “Bank”)],[78015K7F5],[US78015K7F50],"[US $300,000,000]","[August 5, 2022]","[Quarterly on the 5th of each February, May, A..."
7,424B2_d795231d424b2.htm,[],"[September, 00914AAE2]","[US00914AAD46, US00914AAE29]","[ $600,000,000, $500,000,000 Trade Date: Sept...","[ 2.250% per annum, accruing from September16,...","[ Each January15 and July15, beginning on Janu..."
8,424B2_d816260d424b2.htm,[],[02665WDE8],[],"[ $500,000,000]","[ February15, 2022, 0.0933]","[ Each February15, May15, August 15 andNovembe..."
9,424B2_d833231d424b2.htm,[],[95000U2K8],[],[],"[ February11, 2026]","[ Each February11 and August11, commencing Aug..."


In [115]:
c = dict({'a': []})

In [117]:
c['a'].append(4)

In [118]:
c

{'a': [4]}

In [140]:
lis = []

d = {'a': ['hello'], 'b': [1]}
dd = {'a': ['hello']}

In [141]:
d['a'].append(4)
dd['a'].append(4)

In [142]:
lis.append(d)
lis.append(dd)

In [143]:
dfdf = pd.DataFrame(lis)

In [144]:
dfdf

,a,b
0,"[hello, 4]",[1]
1,"[hello, 4]",NaN


In [150]:
for x in dfdf.columns:
    dfdf[x] = [str(value).strip('[]').strip("'") for value in dfdf[x]]
dfdf

,a,b
0,"hello', 4",1
1,"hello', 4",nan


In [68]:
all_data[3][-26:]

'424B2_a19-18959_1424b2.htm'

In [19]:
soup = BS(list_htms[0])
string = StringIO(soup.text)
df_1 = pd.read_csv(string, names=['col'])

In [99]:
soup = BS(list_htms[21])
re.findall(r'(.{0,17}[ACDMI]\w+:\s*[\n]*.{0,40}\d)', soup.text)

['Principal Amount:\nUS $1,500,000,000',
 'Minimum Denominations:\n$2,000 and integral multiples of $1,000',
 'Pricing Date:\nOctober 21, 2019',
 'Issue Date:\nOctober 24, 2019',
 'Maturity Date:\nNovember 1',
 '    Code:\n78015K7C2 / US78015K7C20',
 'Interest Payment Dates:\nSemi-Annually, on each May 1',
 'Underwriting Discount:\n0.250']

In [40]:
# i = 1
# j = 3

# if i:
#     print(i)
#     if j:
#         print(j)

1
3


In [ ]:
# (                # begin capture
#   [A-Z]            # one uppercase letter  \ First Word
#   [a-z]+           # 1+ lowercase letters  /
#   (?=\s[A-Z])      # must have a space and uppercase letter following it
#   (?:                # non-capturing group
#     \s               # space
#     [A-Z]            # uppercase letter   \ Additional Word(s)
#     [a-z]+           # lowercase letter   /
#   )+              # group can be repeated (more words)
# )               #end capture

In [ ]:
# 20
'Key TermsIssuer: GS Finance Corp.\nGuarantor: The Goldman Sachs Group,Inc.Underlying indices: the S&P 500® Index (Bloomberg symbol, “SPX Index”), as published by Standard & Poor’s Financial Services LLC, and the Russell 2000® Index (Bloomberg symbol, “RTY Index”), as published by FTSE Russell; see “The Underlying Indices” on pageS-27 \nSpecified currency: U.S. dollars (“$”)Face amount: each note will have a face amount of $10, or integral multiples of $10 in excess thereof; $2,000,000 in the aggregate for all the offered notes; the aggregate face amount of the offered notes may be increased if the issuer, at its sole option, decides to sell an additional amount of the offered notes on a date subsequent to the date of this prospectus supplement\nDenominations: $10 and integral multiples of $10 in excess thereofMinimum purchase amount:in connection with the initial offering of the notes, the minimum face amount of notes that may be purchased by any investor is $1,000\nSupplemental plan of distribution: GS Finance Corp. will sell to Goldman Sachs & Co. LLC (“GS&Co.”), and GS&Co. will purchase from GS Finance Corp., the aggregate face amount of the offered notes specified on the front cover of this prospectus supplement. GS&Co. proposes initially to offer the notes to the public at the original issue price set forth on the cover page of this prospectus supplement, and to UBS Financial Services Inc. at such price less a concession not in excess of 0.75% of the face amount.'
'\nStated maturity date: October 7, 2024, subject to adjustment as described under “Specific Terms of Your Notes — Stated Maturity Date” on pageS-25Determination date: October 2, 2024, subject to adjustment as described under “Specific Terms of Your Notes — Determination Date” on pageS-26\nNo interest: the notes do not bear interestS-5\n\n\nNo redemption: the notes will not be subject to redemption right or price dependent redemption rightNo listing: the notes will not be listed on any securities exchange or interdealer market quotation system\nCalculation agent: GS&Co.Business day: as described under “Specific Terms of Your Notes — Special Calculation Provisions — Business Day” on pageS-29\nTrading day: as described under “Specific Terms of Your Notes — Special Calculation Provisions — Trading Day” on pageS-29CUSIP no.: 36258L718\nISIN no.: US36258L7183FDIC: the notes are not bank deposits and are not insured by the Federal Deposit Insurance Corporation or any other governmental agency, nor are they obligations of, or guaranteed by, a bank\n\n'

# 22
'DETAILSIssuer:Caterpillar Financial Services CorporationTitle of Securities:Medium-Term Notes, Series I, Floating Rate Notes Due 2021Form of Security:Global NoteFormat:SEC Registered-Registration Statement Number 333-217029Trade Date/Pricing Effective Time:January 9, 2020Settlement Date (Original Issue Date):January 13, 2020Maturity Date:November 12, 2021Principal Amount:$750,000,000Price to Public (Issue Price):100.000%Dealer’s Commission:0.125% (12.5 basis points)All-in-price:99.875%Net Proceeds to Issuer:749062500Interest Rate Basis (Benchmark):3 Month USD LIBOR (except for the Initial Interest Rate, which shall be 1 Month USD LIBOR)Index Currency:U.S. DollarsSpread (Plus or Minus):+20.0 basis points (0.20%)Spread Multiplier:N/ASpread/Spread Multiplier Reset Option:N/AOptional Reset Dates (only applicable if option to reset spread or spread multiplier):N/ABasis for Interest Rate Reset (only applicable if option to reset spread or spread multiplier):N/ASpecified Currency:U.S. DollarsOption to Elect Payment in U.S. Dollars(only applicable'
